# Solving Branch and Bound Problems

## Pulp Library

We will be using Pulp library.

Pulp is Open Source LP modeler written in python. PuLP can generate MPS or LP files and call GLPK, COIN CLP/CBC, CPLEX, and GUROBI to solve linear problems.

### link to pulp documentation
* https://coin-or.github.io/pulp/main/index.html

### PyPi link
* https://pypi.org/project/PuLP/

In [2]:
# check if we have pulp if not we install it
try:
    import pulp
except ImportError:
    print("pulp not installed")
    # install pulp with !pip
    !pip install pulp
    import pulp


pulp not installed
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 69.2 MB/s eta 0:00:00


## Sample Problem

Solve the following BIP be branch and bound.

        Max  Z = 8x1 + 11x2
        s.t.     5x1 + 7x2  <= 14
            0 <= x1 , x2  <= 1    integer

Now we can and will solve this using brute force, since there are only two variables with only two possible values each.
Thus we have 2**2 == 4 possible solutions to check.

In [3]:
# from pulp import * # this is bad practice but we do it for the sake of the tutorial
from pulp import LpProblem, LpMaximize, LpVariable, LpStatus, value   # this is better practice
# using * means we pollute the namespace with all the pulp functions
# this could cause problems if we do this with multiple libraries - huge namespace conflicts

# Create the problem variable
prob = LpProblem("What to do", LpMaximize)

# Define the decision variables as floats
x1 = LpVariable("Breakfast", lowBound=0) # cat default is Continuous so we don't need to specify
x2 = LpVariable("School", lowBound=0)

# Define the objective function
prob += 8*x1 + 11*x2

# Define the constraints
prob += 5*x1 + 7*x2 <= 10 # original constraint 14 would lead to 1, 1 solution which is optimal also for integers
prob += x1 <= 1
prob += x2 <= 1


# Solve the problem
prob.solve()

# Print the optimal solution
print("Optimal life quantities:")
print("Breakfast:", value(x1))
print("School:", value(x2))
print("Total karma:", value(prob.objective))

Optimal life quantities:
Breakfast: 1.0
School: 0.71428571
Total karma: 15.85714281


/usr/local/lib/python3.12/dist-packages/pulp/pulp.py:1489: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


## Brute force

In [4]:
# first let's do a brute force approach we simple plug in all possible values
# here we have 2 variables and 2 constraints so we can do this easily

# we can do this by hand but we can also use the itertools library
import itertools

# we can use the product function to get all possible combinations of 0 and 1
combinations = list(itertools.product([0, 1], repeat=2))
# print combinations
# testing the following combinations
print(f"{'Breakfast':<10}{'School':<10}{'Total karma':<10}")
for breakfast, school in combinations:
    karma = 8*breakfast + 11*school
    print(f"{breakfast:<10}{school:<10}{karma:<10}")

Breakfast School    Total karma
0         0         0         
0         1         11        
1         0         8         
1         1         19        


In [ ]:
# so if we were doing the branch and boudn approach
# first we would calculate the optimal solution for the continuous case
# here it is
# Breakfast: 1.0
# School: 0.71428571
# Total karma: 15.85714281

# then we would branch on the School variable
# and we would get the following two subproblems
# Breakfast: 1.0
# School: 1.0
# this is subproblem 1 - it is infeasible because the constraint 5*x1 + 7*x2 <= 10 is violated

# Breakfast: 1.0
# School: 0.0
# Total karma: 8.0 - this would be lower bound for the second subproblem

# then we would branch on the Breakfast variable
# and we would get the following two subproblems
# Breakfast: 0.0
# School: 0.71428571

# Breakfast: 1.0
# School: 0.71428571

# then we would end up with the following optimal solution
# Breakfast: 0.0
# School: 1.0
# Total karma: 11.0 - this would actually be the optimal solution for the integer case

# here with branch and bound we solved the problem in 3 steps

# in this simple example we gained almost nothing, except we did not have to check the infeasible solution
# but in more complex problems we can save a lot of time by using branch and bound

## Pulp for integers

In [5]:
# Create the problem variable
prob = LpProblem("What to do", LpMaximize)

# Define the decision variables as floats
x1 = LpVariable("Breakfast", lowBound=0, cat="Integer")
x2 = LpVariable("School", lowBound=0, cat="Integer")

# Define the objective function
prob += 8*x1 + 11*x2

# Define the constraints
prob += 5*x1 + 7*x2 <= 10 # original constraint 14 would lead to 1, 1 solution which is optimal also for integers
prob += x1 <= 1
prob += x2 <= 1


# Solve the problem
prob.solve()

# Print the optimal solution
print("Optimal life quantities:")
print("Breakfast:", value(x1))
print("School:", value(x2))
print("Total karma:", value(prob.objective))

Optimal life quantities:
Breakfast: 0.0
School: 1.0
Total karma: 11.0


## Multiple Binary Variables

In [6]:
# now let's come up with a more complex example
# let's stick with binary choices
# but let's have say 8 decisions to make - yes or no

# so the 8 choices will be 8 different ice cream flavors
# there will be a taste function that will tell us how much we like each flavor
# we want to optimize the total taste
# finally the constraint will be that no more than 5 varieties can be chosen - so we don't get sick

# first let's come up with taste function
# we will use a dictionary to store the taste of each flavor
taste = {
    "vanilla": 5,
    "chocolate": 7,
    "strawberry": 3,
    "mint": 4,
    "caramel": 6,
    "pistachio": 8,
    "banana": 2,
    "coconut": 9
}

# now let's create the problem
prob = LpProblem("Ice cream", LpMaximize)

# Define the decision variables as floats
# we will use a dictionary to store the variables
flavors = LpVariable.dicts("Flavor", taste.keys(), cat="Binary")

# Define the objective function
prob += sum(taste[flavor] * flavors[flavor] for flavor in flavors)

# Define the constraints
prob += sum(flavors[flavor] for flavor in flavors) <= 5
# let's add the constraint that we can't have vanilla and chocolate at the same time
prob += flavors["vanilla"] + flavors["chocolate"] <= 1


# turn off verbose - we would get too much output
pulp.LpSolverDefault.msg = 0

# Solve the problem
prob.solve()

# let's print the optimal solution
print("Optimal ice cream choices:")
for flavor in flavors:
    if value(flavors[flavor]) == 1:
        print(flavor, taste[flavor])

# let's print the total taste
print("Total taste:", value(prob.objective))

Optimal ice cream choices:
chocolate 7
mint 4
caramel 6
pistachio 8
coconut 9
Total taste: 34.0


In [ ]:
# note with some minimal constraints we could have used the greedy algorithm
# simply start with the best flavor (highest linear coefficient) and keep adding flavors until we reach the constraint

# in general case this is not going to work
# we could have some complex constraints that would make the greedy algorithm fail
# we pick some flavors that are not optimal just to satisfy the constraint

In [ ]:
# now let's do the same problem with same flavor values but we will allow for variables to have 3 values
# 0 - no flavor, 1 - some flavor, 2 - full flavor
# we will also add a constraint that we can't have more than 4 flavors at the same time

# Create the problem variable
prob = LpProblem("Ice_cream", LpMaximize)


# we will use a dictionary to store the variables
flavors = LpVariable.dicts("Flavor", taste.keys(), lowBound=0, upBound=2, cat="Integer")
# so lowBound is 0 and upBound is 2 those are integer values and are inclusive!

# Define the objective function
prob += sum(taste[flavor] * flavors[flavor] for flavor in flavors)

# Define the constraints

# we can't have more than 4 flavors at the same time
prob += sum(flavors[flavor] for flavor in flavors) <= 4 # we can have 4 half flavors or 2 full flavors
# or we could have 1 full flavor and 2 half flavors

# let's add the constraint that we can't have vanilla and chocolate at the same time
prob += flavors["vanilla"] + flavors["chocolate"] <= 2 # this means we can go half vanilla and half chocolate
# let's add constraint that pistachio and coconut can't be chosen at the same time - they are too similar
prob += flavors["pistachio"] + flavors["coconut"] <= 2

# Solve the problem
prob.solve()

# let's print the optimal solution
print("Optimal ice cream choices:")
for flavor in flavors:
    if value(flavors[flavor]) > 0:
        print(flavor, taste[flavor])

# let's print the total taste
print("Total taste:", value(prob.objective))


Optimal ice cream choices:
chocolate 7
coconut 9
Total taste: 32.0


In [ ]:
# again solution here was pretty obvious
# we got coconut and chocolate as full flavors
# a greedy algorithm would have worked here as well

# TODO think of a more complex example where greedy algorithm would fail

In [ ]:
# let's do a more complex example
# we will use the existing flavors but we will also have two continuous variables
# we will have a budget of 15 and we will have to buy the ice cream
# each flavor will have a price

# we will have to optimize the total taste

# Create the problem variable

prob = LpProblem("Ice_cream", LpMaximize)

# Define the decision variables as floats
# we will use a dictionary to store the variables
flavors = LpVariable.dicts("Flavor", taste.keys(), lowBound=0, upBound=2, cat="Integer")

# Define the decision variables as floats
# we will use a dictionary to store the variables
prices = {
    "vanilla": 1,
    "chocolate": 2,
    "strawberry": 3,
    "mint": 4,
    "caramel": 5,
    "pistachio": 6,
    "banana": 7,
    "coconut": 8
}

# add the continuous variables
# budget is up to 15
budget = LpVariable("Budget", lowBound=0, upBound=15) # continuous variable

# Define the objective function
prob += sum(taste[flavor] * flavors[flavor] for flavor in flavors)

# Define the constraints
prob += sum(prices[flavor] * flavors[flavor] for flavor in flavors) <= budget

# we can't have more than 4 flavors at the same time
prob += sum(flavors[flavor] for flavor in flavors) <= 4 # we can have 4 half flavors or 2 full flavors

# let's add the constraint that we can't have vanilla and chocolate at the same time
prob += flavors["vanilla"] + flavors["chocolate"] <= 2 # this means we can go half vanilla and half chocolate
# let's add constraint that pistachio and coconut can't be chosen at the same time - they are too similar
prob += flavors["pistachio"] + flavors["coconut"] <= 2

# Solve the problem
prob.solve()

# let's print the optimal solution
print("Optimal ice cream choices:")
print(f"{'Flavor':<10}{'Taste':<10}{'Amount':<10}{'Price kg':<10}{'Cost':<10}")
for flavor in flavors:
    if value(flavors[flavor]) > 0:
        print(f"{flavor:<10}{taste[flavor]:<10}{value(flavors[flavor]):<10}{prices[flavor]:<10}{prices[flavor]*value(flavors[flavor]):<10}")
        # print(f"{flavor:<10}{taste[flavor]:<10}{value(flavors[flavor]):<10}")
        # print(flavor, taste[flavor])

# let's print the total taste
print("Total taste:", value(prob.objective))

Optimal ice cream choices:
Flavor    Taste     Amount    Price kg  Cost      
chocolate 7         2.0       2         4.0       
caramel   6         1.0       5         5.0       
pistachio 8         1.0       6         6.0       
Total taste: 28.0
